This tutorial gives an overview of the functionality of *powfacpy*. For a complete list of classes and methods, please have a look at the *API* Chapter of the documentation or at the source code.

Similar to using the Python API of *PowerFactory* directly, we first need to import the powerfactory module from *PowerFactory*'s installation directory and get the application.

In [5]:
import sys
sys.path.append(r'C:\Program Files\DIgSILENT\PowerFactory 2022 SP1\Python\3.10') # you may use a different directory
import powerfactory as powerfactory
app = powerfactory.GetApplication()

Then import *powfacpy* and create an instance of class `PFBaseInterface` (with argument `app`). This interface class is used mainly to interact with the *PowerFactory* database as we will see below. 

In [6]:
import powerfactorypy
pfbi = powerfactorypy.PFBaseInterface(app)

Note that `pfbi` has an attribute `app` that can be used similar to the `app` variable we loaded from the `powerfactory` module before. Here are two ways to 1. show the PowerFactory application and 2. activate a project:

In [7]:
app.Show()
app.ActivateProject(r"\user\powerfactorypy_base") # You may change the project path
pfbi.app.Show()
pfbi.app.ActivateProject(r"\user\powerfactorypy_base") # You may change the project path

1

So where does `pfbi` differ from the Python interface that `app` provides? Let's see.

# Accessing objects
 Let's access an object from the *PowerFactory* database. When using `app`, we would have to write several lines of code using the methods `app.GetProjectFolder`, `app.GetChildren` or `app.GetContents`. In contrast, it is much easier to use `pfbi.get_obj` with the path of the object (**relative to the project folder**) as an argument: 

In [8]:
terminal_mv = pfbi.get_obj(r"Network Model\Network Data\Grid\Terminal MV")

Note that you can easily copy and paste the path from the data manager while selecting the object in the data tree: 

![object path](object_path.png)

# Setting and getting object attributes
How about setting data in the database? You can set attributes of an object as follows:

In [9]:
pfbi.set_attr(r"Network Model\Network Data\Grid\Terminal MV",
    {"uknom":33,"outserv":0})

So with only one command we set the attributes "uknom" and "outserv" of the terminal. This saves time and is also very readable code!
Note that the method `set_attr` accepts the path (string) but also a *PowerFactory* object. For example, we could also use the object `terminal_mv` that we loaded above:

In [10]:
pfbi.set_attr(terminal_mv,
    {"uknom":33,"outserv":0})

This applies to many other methods in powfacpy. Loading the object only once and then using the object can be more efficient than using the path string very many times.

If you want to get an attribute of an object, write: 

In [ ]:
nominal_voltage = pfbi.get_attr(r"Network Model\Network Data\Grid\Terminal MV","uknom")

# Accessing objects from a folder
There are other functions to get objects from the database, for example:

In [11]:
terminals_110kV = pfbi.get_from_folder(r"Network Model\Network Data\Grid",obj_name="*.ElmTerm",
    attr="uknom", attr_lambda=lambda x : x==110, include_subfolders=True)

This returns a list of all the terminals in the folder "Network Model\Network Data\Grid" (and its subfolders) whose nominal voltage (attribute `uknom`) is 110 kV. The `obj_name` can contain placeholders. `attr_lambda` must be a lambda function that returns `True` if `attr` has a certain condition (e.g. if you want to get all terminals with nominal voltage below 60 kV, you could also use `attr_lambda=lambda x : x<60`.

Note that except for the folder, these are all optional method arguments. To get all objects in the grid folder for example, you can write:

In [12]:
all_grid_objects = pfbi.get_from_folder(r"Network Model\Network Data\Grid")

# Creating objects
If you want to create a new object, use:

In [13]:
new_obj = pfbi.create_by_path(r"Library\Dynamic Models\dummy.BlkDef") 

This will create an object of class "BlkDef" with the name "dummy" in the folder "Library\Dynamic Models". You can also use

In [14]:
new_obj = pfbi.create_in_folder(r"Library\Dynamic Models","dummy.BlkDef",overwrite=True)

which will overwrite the former object.

# Copying objects
To copy an object to a folder use

In [15]:
obj_to_be_copied = r"Library\Dynamic Models\Linear_interpolation"
folder_copy_to = r"Library\Dynamic Models\TestCopy"
new_copy = pfbi.copy_obj(obj_to_be_copied,folder_copy_to)

As mentioned above, insted of using objects as method arguments, you can also use the path string:

In [16]:
new_copy = pfbi.copy_obj(r"Library\Dynamic Models\Linear_interpolation",
    r"Library\Dynamic Models\TestCopy")

To copy a whole folder, write:

In [18]:
folder_copy_from = r"Library\Dynamic Models\TestDummyFolder"
folder_copy_to = r"Library\Dynamic Models\TestCopyMultiple"
new_copy_list = pfbi.copy_multiple_objects(folder_copy_from,folder_copy_to)

You can also copy a list of objects. To show this, we get the contents of a folder first and then copy them to the new folder:

In [20]:
list_of_objects = pfbi.get_from_folder(r"Library\Dynamic Models\TestDummyFolder")
folder_copy_to = r"Library\Dynamic Models\TestCopyMultiple"
new_copy_list = pfbi.copy_multiple_objects(list_of_objects,folder_copy_to)

# Deleting objects
If you want to delete an object, you can simply do:

In [21]:
pfbi.delete_obj(r"Library\Dynamic Models\TestCopyMultiple\dummy")

Alternatively, to delete multiple objects (use placeholder "*") in a folder:

In [ ]:
folder = r"Library\Dynamic Models\TestDelete"
pfbi.delete_obj_from_folder(folder,"dummy_to_be_deleted*")